## Preamble

In [7]:
from example_helpers import setup_notebook
from pyetm.models import Scenario

setup_notebook()

Environment setup complete
  Using ETM API at     http://localhost:3000/api/v3
  Token loaded?        True
API connection ready


## Picking my scenario

Use an API session ID to load up one of your scenarios.


In [8]:
# Example beta scenario
# scenario = Scenario.load(2690506)

# Example pro scenario
scenario = Scenario.load(2690288)

## Create a Packer

The scenario packer allows you to collect scenarios and define what information you want to extract from all of them. It helps to collect all info into dataframes or Excel.

- **Main info** with `main_info()` you display all meta information about the scenarios, like area code and end years;

- Input settings
    - **Inputs** `inputs()` displays all inputs for the scenario including their unit, default and user-set values;
    - **Custom curves** using `custom_curves()` you can retrieve and display the curves uploaded to the scenario;
    - **Sortables** `sortables()` shows all user sortables set for the scenario, like heat network orders, etc;
- Outputs:
    - **GQuery results** with `gquery_results()` you can retrieve the queries that were prepared for the scenarios;
    - **Carrier curves** `carrier_curves()` downloads and shows the carreir curves requested for the scenarios.


Scenarios can be added to a packer in multiple ways:

- `add(*scenarios)`  will ready the scenario for all 6 packs (or Excel tabs) described above
- `add_inputs(*scenarios)` will only ready the scenarios for the inputs dataframe or Excel tab
- `add_sortables(*scenarios)` will only ready the scenarios for the sortables dataframe or Excel tab
- `add_custom_curves(*scenarios)` will only ready the scenarios for the custom_curves dataframe or Excel tab
- `add_carrier_curves(*scenarios)` will only ready the scenarios for the custom_curves dataframe or Excel tab

If a scenario is added in any of the individual packs, it will be automatically added for main info and prepared gqueries.

-----------------

NOTE: the packer currently only works for a single scenario!

-----------------

In [9]:
from pyetm.models import ScenarioPacker

packer = ScenarioPacker()

# Ready the scenario for inputs
packer.add_inputs(scenario)

scenario2 = Scenario.load(2690509)
packer.add_inputs(scenario2)

# Main info is automatically readied
print(packer.inputs())
print(packer.main_info())

scenario                                            unit 2690509             \
                                                           value    default   
input                                                                         
agriculture_bio_kerosene_in_crude_oil_share            %     0.0        0.0   
agriculture_biodiesel_in_crude_oil_share               %     0.0  53.562837   
agriculture_burner_crude_oil_share                     %     0.0   0.522009   
agriculture_burner_hydrogen_share                      %    20.0        0.0   
agriculture_burner_network_gas_share                   %     0.0  34.320411   
...                                                  ...     ...        ...   
wtp_of_industry_chemicals_other_flexibility_p2h...  euro    25.0       22.0   
wtp_of_industry_chemicals_refineries_flexibilit...  euro    25.0       22.0   
wtp_of_industry_other_food_flexibility_p2h_elec...  euro    25.0       22.0   
wtp_of_industry_other_paper_flexibility_p2h_ele...  

When you add gqueries to a scenario which is connected to the packer, these queries will be readied for the pack.

In [10]:
scenario.add_queries(
    ["dashboard_total_costs", "dashboard_co2_emissions_versus_start_year"]
)

import pandas as pd
cols = [scenario.id for scenario in packer._all_scenarios()]
data = pd.concat(
    [scenario.results() for scenario in packer._all_scenarios()],
    axis=1,
    keys=cols
)

# col = [col for col in data.columns]
print(data.columns.get_level_values(0))

# print(packer.gquery_results())

Index([2690288, 2690288, 2690288], dtype='int64')


Queries can be added on-the-go. When the packer is exported to Excel, all requested queries will show up.

In [11]:
scenario2.add_queries(
    ["dashboard_emissions"]
)

packer.to_excel('my_excel.xlsx')

# Create multiple packers



In [12]:
# Load multiple scenarios for comparison
scenario_a = Scenario.load(2690506)  # These are just random at the moment
scenario_b = Scenario.load(2690288)
scenario_c = Scenario.load(2690503)

# Make a new packer
packer = ScenarioPacker()

# Get generic meta info for all scenarios in the packer
main_df = packer.main_info()
print(main_df)

Empty DataFrame
Columns: []
Index: []


In [13]:
# gqueries to all scenarios
gqueries = [
    "final_demand_from_industry_energetic",
    "final_demand_from_transport_energetic",
    "final_demand_from_households_energetic",
    "dashboard_total_costs",
    "dashboard_co2_emissions_versus_start_year"
]
for scenario in [scenario_a, scenario_b, scenario_c]:
    scenario.add_queries(gqueries)

# add inputs
packer.add_inputs(scenario_a, scenario_b, scenario_c)

gquery_df = packer.gquery_results()
inputs_df = packer.inputs()

print(f"Gqueries:      ")
print(gquery_df.head(10))
print(f"Inputs:      ")
print(inputs_df.head(10))

Gqueries:      
                                               unit     2690288     2690503  \
gquery                                                                        
dashboard_co2_emissions_versus_start_year    factor   -1.004340   -0.324385   
dashboard_total_costs                      bln_euro   54.600271   63.523249   
final_demand_from_households_energetic           PJ  245.072081  349.944243   
final_demand_from_industry_energetic             PJ  525.206828  696.221878   
final_demand_from_transport_energetic            PJ  264.130299  464.359945   

                                              2690506  
gquery                                                 
dashboard_co2_emissions_versus_start_year   -1.026581  
dashboard_total_costs                       64.227005  
final_demand_from_households_energetic     258.986634  
final_demand_from_industry_energetic       815.856409  
final_demand_from_transport_energetic      290.607481  
Inputs:      
scenario                

In [14]:
# To excel with multiple scenarios
packer.to_excel('multi_scenario_demo.xlsx')

In [15]:
# You can also just pack parts of a scenario
advanced_packer = ScenarioPacker()

# Add scenarios selectively for different data types
advanced_packer.add_inputs(scenario_a, scenario_b)  # Only A and B for inputs
advanced_packer.add_custom_curves(scenario_c)       # Only C for custom curves

print(advanced_packer.get_summary())

{'total_scenarios': 3, 'custom_curves_count': 1, 'inputs_count': 2, 'sortables_count': 0, 'carrier_curves_count': 0, 'scenario_ids': [2690288, 2690503, 2690506]}
